In [1]:
import pandas as pd
from IPython.display import display
df = pd.read_csv('./data/driving_log.csv')
display(df.head(n=5))

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
3,IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
4,IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [24]:
from sklearn.cross_validation import train_test_split
df_train, df_val = train_test_split(df, test_size=0.2)

In [47]:
import numpy as np
#print(df.shape)
#print(df.count())
test = df.iloc[0:4]
for index, row in test.iterrows():
    print(os.path.join('./data', row[CENTER]))

./data/IMG/center_2016_12_01_13_30_48_287.jpg
./data/IMG/center_2016_12_01_13_30_48_404.jpg
./data/IMG/center_2016_12_01_13_31_12_937.jpg
./data/IMG/center_2016_12_01_13_31_13_037.jpg


In [23]:
CENTER = "center"
LEFT = "left"
RIGHT = "right"
STEERING = "steering"

In [50]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
def generator(df, batch_size=32):
    num_samples = df.shape[0] #or df['center'].count()
    while 1:
        shuffle(df) # or df.sample(frac=1).reset_index(drop=True)
        for offset in range(0, num_samples, batch_size):
            df_batch = df.iloc[offset:offset + batch_size] #slice wont raise out of bound exception
            images = []
            angles = []
            for index, row in df_batch.iterrows():
                center_image = cv2.imread(os.path.join('./data', row[CENTER]))
                center_image = cv2.cvtColor(center_image, cv2.COLOR_BGR2RGB)
                angle = row[STEERING]
                images.append(center_image)
                angles.append(angle)
            X_train = np.array(images)
            y_train = np.array(angles)
            yield X_train, y_train

In [25]:
from keras.models import Sequential
from keras.layers import Input, Lambda
from keras.layers import Dropout, Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, Cropping2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [30]:
model = Sequential()
model.add(Cropping2D(cropping=((40, 40), (0, 0)), input_shape=(160, 320, 3)))
model.add(BatchNormalization())
model.add(Conv2D(96, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(384, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(192, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(1))

In [31]:
callbacks = [EarlyStopping(monitor='val_loss', patience=30, verbose=0),
             ModelCheckpoint('model.hdf5', monitor='val_loss', save_best_only=True, verbose=0)]

In [53]:
EPOCHS = 20
adam_opt=Adam(lr=5e-5)
model.compile(adam_opt, 'mse', ['accuracy'])
model_history = model.fit_generator(generator(df_train), epochs=EPOCHS,
                                    steps_per_epoch=int(df_train.shape[0]/32),
                                    validation_data=generator(df_val),
                                    validation_steps=int(df_val.shape[0]/32),
                                    callbacks = callbacks)

Epoch 1/20
200/200 [==============================] - 15s - loss: 0.0082 - acc: 0.5427 - val_loss: 0.0113 - val_acc: 0.5413
Epoch 2/20
200/200 [==============================] - 15s - loss: 0.0080 - acc: 0.5424 - val_loss: 0.0114 - val_acc: 0.5457
Epoch 3/20
200/200 [==============================] - 15s - loss: 0.0080 - acc: 0.5433 - val_loss: 0.0112 - val_acc: 0.5457
Epoch 4/20
200/200 [==============================] - 15s - loss: 0.0077 - acc: 0.5427 - val_loss: 0.0112 - val_acc: 0.5431
Epoch 5/20
200/200 [==============================] - 15s - loss: 0.0077 - acc: 0.5421 - val_loss: 0.0117 - val_acc: 0.5400
Epoch 6/20
200/200 [==============================] - 15s - loss: 0.0072 - acc: 0.5426 - val_loss: 0.0108 - val_acc: 0.5400
Epoch 7/20
200/200 [==============================] - 15s - loss: 0.0072 - acc: 0.5427 - val_loss: 0.0116 - val_acc: 0.5438
Epoch 8/20
200/200 [==============================] - 15s - loss: 0.0067 - acc: 0.5433 - val_loss: 0.0114 - val_acc: 0.5438
Epoch 9/